# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")
```

env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]
print("Brain Name: ", brain_name)
# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print("State sizes: ", states.shape)
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

### 4.1 Import settings
Initiate the environment

In [1]:
import os
import torch 
import random
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
from unityagents import UnityEnvironment
import numpy as np
from maddpg_N import *
# !pip install progressbar2
import progressbar as pb
from torch.utils.tensorboard import SummaryWriter

#---------------Environment set-up--------------------#
env =UnityEnvironment(file_name='Tennis_Linux_NoVis/Tennis.x86_64')
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# Initiate environment
env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]            # get the current state
score = 0

# number of agents in the environment
n_agents = len(env_info.agents)
print('Number of agents:', n_agents)
# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)
state_size = len(state)
print('States have length:', state_size)
#agent = DDPGAgent(state_size=state_size, action_size=action_size, seed=0)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Number of agents: 2
Number of actions: 2
States have length: 24


ddpg_body_dim = [(256,128),(256,128)]

maddpg = MADDPG(state_size, action_size, ddpg_body_dim=ddpg_body_dim, seed=0, batch_size=128, buffer_size=int(10e5), tau=1e-3,\
                gamma=0.99, decay_noise=0.99,lr_actor=1e-3, lr_critic=1e-3, weight_decay=1e-6, device=device)

In [ ]:
def main():
    """
    Multiagent Deep deterministic Policy Gradient (MADDPG)
    This specific MMADDPG will learn how to play a game of tennis. There are 24 states-observables and 2 action-dimensions for
    each agent. 
        
    """
    # ---------------Configuration----------------#
     # Episodes and Trajectory
    n_episodes = 10000
    trajectory_length = 1000
    batch_size = 256
    
    # Model Parameters
    ddpg_body_dim = [(400,300),(400,300)]
    seed = 10
    buffer_size = int(1e5)
    tau = 1e-3
    gamma = 0.995
    decay_noise = 0.98
    lr_actor = 5e-4
    lr_critic = 1e-3
    weight_decay = 0
    multiple_learning = 3
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
    print("Device used: ", device)
    maddpg = MADDPG(state_size, action_size, n_agents, linear_net_dim=ddpg_body_dim, seed=seed,\
                    batch_size=batch_size, buffer_size=buffer_size, tau=tau, gamma=gamma,\
                    decay_noise=decay_noise,lr_actor=lr_actor, lr_critic=lr_critic,\
                    weight_decay=weight_decay, device=device)
    
    # Save configurations
    log_path = os.getcwd()+"/log"
    model_dir= os.getcwd()+"/model_dir"
    os.makedirs(model_dir, exist_ok=True)
    
    # initialize logging of rewards
    logger = SummaryWriter(log_dir=log_path)
    agent0_reward = deque(maxlen=100)
    agent1_reward = deque(maxlen=100)

    all_scores = []
    all_avg_scores = []
    scores_100 = deque(maxlen=100)    
    
    widget = ["Training Status: ", pb.Timer(), ' ', pb.Percentage(), ' ', pb.ETA(), ' ', pb.Bar()]
    #timer = pb.progressbar(widgets=widget, maxval=n_episodes).start()
    
    ##############################################################################
    #---------------------------Training Commence--------------------------------#
    for i_episode in pb.progressbar(range(1, n_episodes+1), widgets=widget):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        states = env_info.vector_observations           # get the current state
        scores = np.zeros(len(env_info.agents))
        for t in range(trajectory_length):
        
            actions = maddpg.act(states, noise=1.0)  # state[n_agent,dim], noise=1.0 (on)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations   # get the next state: array -  (num_agent, state_feature_dim)
            rewards = env_info.rewards                   # get the reward: list - [num_agent]
            dones = env_info.local_done                  # see if episode has finished: list - [num_agent]
            
            # Add to memory
            if (np.any(dones)): 
                maddpg.memory.add_experience(states,actions,rewards, next_states,dones)
            scores += rewards                               # update the score
            states = next_states                             # roll over the state to next time step
            if np.any(dones) and t>100:
                break
            
            
            if (len(maddpg.memory) > batch_size) and (i_episode % 4 == 0):
                for i in range(multiple_learning):
                    for agent_i in range(2):
                        maddpg.step(batch_size, agent_i, logger)
                    maddpg.update_targets()
        # LR Update once after every episode
        maddpg.lr_update()                 
           
        agent0_reward.append(scores[0])       # save latest score into a variable containing the 100 latest scores
        agent1_reward.append(scores[1])
        all_scores.append(np.max(scores))
        scores_100.append(np.max(scores))
        avg_score = np.mean(scores_100)
        all_avg_scores.append(avg_score)
                

        if i_episode % 100 ==0 or i_episode==n_episodes-1:
            avg_rewards = [np.mean(agent0_reward), np.mean(agent1_reward)]
            for agent_i , avg_reward in enumerate(avg_rewards):
                logger.add_scalar("Agent%i/mean_episode_rewards"% agent_i, avg_reward, i_episode)
            
        if i_episode % 100 == 0:
            print('\rEpisode {}\tScore: {}\t 100 Moving Average Score: {}, OUNOISE: {}, pl {}, cl {} '.format(i_episode, scores, avg_score, maddpg.noise.state*maddpg.decay_noise, maddpg.pl, maddpg.cl))
            
        # save model
        agent_checkpoints = []
        if np.mean(avg_score)>=0.5 and i_episode >=100:
            
            print('\nEnvironment solved in {:d} episodes!\t Score: {:5.2f}\t 10 Moving Average Score: {:.2f}'.format(i_episode, score, avg_score))
            for agent_i in range(2):
                checkpoint = {
                    'net_params': maddpg.maddpg_agent[agent_i].network_local.state_dict()}   
                agent_checkpoints.append(checkpoint)
            print("\nModel saved!💎🙌")
            torch.save(agent_checkpoints, model_dir+'checkpoint.pt')
    logger.close()
    
    return (all_scores, all_avg_scores, maddpg)

output_tuple = main()

Device used:  cuda:0


Training Status: Elapsed Time: 0:03:43   1% ETA:   6:08:32 |                  |

Episode 100	Score: [-0.04 -0.02]	 100 Moving Average Score: -0.011699999179691077, OUNOISE: [0.06166402 0.06024063], pl 0.005723802372813225, cl 2.576050803781982e-07 


Training Status: Elapsed Time: 0:08:27   2% ETA:   6:54:46 |                  |

Episode 200	Score: [ 0.07 -0.04]	 100 Moving Average Score: 0.15450000317767262, OUNOISE: [0.00707447 0.0074883 ], pl 0.005319547839462757, cl 1.0012987559093744e-06 


...

Training Status: Elapsed Time: 0:32:39   7% ETA:   7:13:54 |#                 |

Episode 700	Score: [-0.02 -0.04]	 100 Moving Average Score: 0.010800001211464405, OUNOISE: [1.04262554e-07 9.79140006e-08], pl 0.0053086839616298676, cl 5.519170827028574e-07 


Training Status: Elapsed Time: 0:37:28   8% ETA:   7:10:58 |#                 |

Episode 800	Score: [-0.06 -0.02]	 100 Moving Average Score: 0.057300001848489046, OUNOISE: [1.20745975e-08 6.35929206e-09], pl 0.00592387281358242, cl 1.1788645224442007e-06 


Training Status: Elapsed Time: 0:42:05   8% ETA:   7:06:41 |#                 |

In [ ]:
# plot the scores
import seaborn as sns
scores, avg_scores, _  = output_tuple
sns.set()
fig = plt.figure(figsize=(10,6))
ax1 = fig.add_subplot(121)
ax1.plot(np.arange(len(scores)), scores)
ax2 = fig.add_subplot(122)
ax2.plot(np.arange(len(avg_scores)), avg_scores)

ax1.set(ylabel='Scores', xlabel='Episode #', title="Average agents scores per Episode")
ax2.set(ylabel='Scores', xlabel='Episodes x100#', title="Average agents scores per Episode")
plt.show()
fig.tight_layout()
fig.savefig("/home/lew/Projects/Continuous-control_/plots/20agents-DDPG_reacher.png")

In [ ]:
output_tuple

In [ ]:
np.maximum(np.array((2,0.23)),2.5)